Reference:
https://github.com/umbertogriffo/Predictive-Maintenance-using-LSTM/blob/master/src/lstm/regression.py

In [42]:
import keras

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Activation, Conv1D, MaxPooling1D
%matplotlib inline

In [44]:
jet = pd.read_table('../train_data.txt',sep=' ',header=None)
jet.drop([26,27],axis=1,inplace=True)
jet.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
train_df = jet
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


In [45]:
td = pd.read_table('../test_data.txt',sep=' ',header=None)
td.drop([26,27],axis=1,inplace=True)
td.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
td.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,9.9987,0.2502,100.0,489.05,605.03,1497.17,1304.99,10.52,...,371.69,2388.18,8114.10,8.6476,0.03,369,2319,100.00,28.42,17.1551
1,1,2,20.0026,0.7000,100.0,491.19,607.82,1481.20,1246.11,9.35,...,315.32,2388.12,8053.06,9.2405,0.02,364,2324,100.00,24.29,14.8039
2,1,3,35.0045,0.8400,100.0,449.44,556.00,1359.08,1128.36,5.48,...,183.04,2387.75,8053.04,9.3472,0.02,333,2223,100.00,14.98,8.9125
3,1,4,42.0066,0.8410,100.0,445.00,550.17,1349.69,1127.89,3.91,...,130.40,2387.72,8066.90,9.3961,0.02,332,2212,100.00,10.35,6.4181
4,1,5,24.9985,0.6213,60.0,462.54,536.72,1253.18,1050.69,7.05,...,164.56,2028.05,7865.66,10.8682,0.02,305,1915,84.93,14.31,8.5740


In [46]:
td.groupby('id').max()['cycle'].min()

21

### Kmeans

In [47]:
from sklearn import cluster

In [48]:
k_cluster = cluster.KMeans(n_clusters=6)

In [49]:
def max_min_normalize(df):
    return (df-df.min())/(df.max()-df.min())

In [50]:
feature = jet[['setting1','setting2','setting3']]
test_feature = td[['setting1','setting2','setting3']]

In [72]:
train_data = np.array(max_min_normalize(feature))
test_data = np.array(max_min_normalize(test_feature))

In [73]:
k_cluster.fit(train_data)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [74]:
train_mode = pd.DataFrame(k_cluster.labels_)
test_mode = pd.DataFrame(k_cluster.predict(test_data))

In [75]:
train_mode

,0
0,5
1,3
2,2
3,3
4,2
...,...
53754,0
53755,4
53756,2
53757,2


In [76]:
train_df['mode1']=0
train_df['mode2']=0
train_df['mode3']=0
train_df['mode4']=0
train_df['mode5']=0
train_df['mode6']=0
train_df.loc[train_mode[0]==0,['mode1','mode2','mode3','mode4','mode5','mode6']]=[1,0,0,0,0,0]
train_df.loc[train_mode[0]==1,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,1,0,0,0,0]
train_df.loc[train_mode[0]==2,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,1,0,0,0]
train_df.loc[train_mode[0]==3,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,0,1,0,0]
train_df.loc[train_mode[0]==4,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,0,0,1,0]
train_df.loc[train_mode[0]==5,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,0,0,0,1]

In [77]:
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s18,s19,s20,s21,mode1,mode2,mode3,mode4,mode5,mode6
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,2223,100.00,14.73,8.8071,0,0,0,0,0,1
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,2212,100.00,10.41,6.2665,0,0,0,1,0,0
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,1915,84.93,14.08,8.6723,0,0,1,0,0,0
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,2212,100.00,10.59,6.4701,0,0,0,1,0,0
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,1915,84.93,14.13,8.5286,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,20.0037,0.7000,100.0,491.19,608.79,1495.60,1269.51,9.35,...,2324,100.00,24.36,14.5189,1,0,0,0,0,0
53755,260,313,10.0022,0.2510,100.0,489.05,605.81,1514.32,1324.12,10.52,...,2319,100.00,28.10,16.9454,0,0,0,0,1,0
53756,260,314,25.0041,0.6200,60.0,462.54,537.48,1276.24,1057.92,7.05,...,1915,84.93,14.19,8.5503,0,0,1,0,0,0
53757,260,315,25.0033,0.6220,60.0,462.54,537.84,1272.95,1066.30,7.05,...,1915,84.93,14.05,8.3729,0,0,1,0,0,0


In [78]:
td['mode1']=0
td['mode2']=0
td['mode3']=0
td['mode4']=0
td['mode5']=0
td['mode6']=0
td.loc[test_mode[0]==0,['mode1','mode2','mode3','mode4','mode5','mode6']]=[1,0,0,0,0,0]
td.loc[test_mode[0]==1,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,1,0,0,0,0]
td.loc[test_mode[0]==2,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,1,0,0,0]
td.loc[test_mode[0]==3,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,0,1,0,0]
td.loc[test_mode[0]==4,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,0,0,1,0]
td.loc[test_mode[0]==5,['mode1','mode2','mode3','mode4','mode5','mode6']]=[0,0,0,0,0,1]

In [79]:
td

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s18,s19,s20,s21,mode1,mode2,mode3,mode4,mode5,mode6
0,1,1,9.9987,0.2502,100.0,489.05,605.03,1497.17,1304.99,10.52,...,2319,100.00,28.42,17.1551,0,0,0,0,1,0
1,1,2,20.0026,0.7000,100.0,491.19,607.82,1481.20,1246.11,9.35,...,2324,100.00,24.29,14.8039,1,0,0,0,0,0
2,1,3,35.0045,0.8400,100.0,449.44,556.00,1359.08,1128.36,5.48,...,2223,100.00,14.98,8.9125,0,0,0,0,0,1
3,1,4,42.0066,0.8410,100.0,445.00,550.17,1349.69,1127.89,3.91,...,2212,100.00,10.35,6.4181,0,0,0,1,0,0
4,1,5,24.9985,0.6213,60.0,462.54,536.72,1253.18,1050.69,7.05,...,1915,84.93,14.31,8.5740,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33986,259,119,35.0015,0.8403,100.0,449.44,555.56,1366.01,1129.47,5.48,...,2223,100.00,14.94,8.9065,0,0,0,0,0,1
33987,259,120,42.0066,0.8405,100.0,445.00,549.42,1351.13,1123.86,3.91,...,2212,100.00,10.57,6.4075,0,0,0,1,0,0
33988,259,121,42.0061,0.8400,100.0,445.00,549.65,1349.14,1118.91,3.91,...,2212,100.00,10.57,6.4805,0,0,0,1,0,0
33989,259,122,0.0024,0.0003,100.0,518.67,642.58,1589.61,1408.16,14.62,...,2388,100.00,39.08,23.3589,0,1,0,0,0,0


### 数据处理

In [81]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.loc[train_df['RUL']>130,['RUL']]=130
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s19,s20,s21,mode1,mode2,mode3,mode4,mode5,mode6,RUL
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,100.00,14.73,8.8071,0,0,0,0,0,1,130
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,100.00,10.41,6.2665,0,0,0,1,0,0,130
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,84.93,14.08,8.6723,0,0,1,0,0,0,130
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,100.00,10.59,6.4701,0,0,0,1,0,0,130
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,84.93,14.13,8.5286,0,0,1,0,0,0,130


In [84]:
 train_df.to_csv('./data/processed_train_data.csv')

In [85]:
# normalize the columns in the training data.
# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s20,s21,mode1,mode2,mode3,mode4,mode5,mode6,RUL,cycle_norm
0,1,1,0.833134,0.997625,1.0,0.060269,0.181576,0.311201,0.273095,0.146592,...,0.156036,0.159082,0.0,0.0,0.0,0.0,0.0,1.0,130,0.000000
1,1,2,0.999767,0.998575,1.0,0.000000,0.131847,0.296600,0.245535,0.000000,...,0.007888,0.014562,0.0,0.0,0.0,1.0,0.0,0.0,130,0.002653
2,1,3,0.595096,0.738480,0.0,0.238089,0.016332,0.035297,0.056997,0.293184,...,0.133745,0.151414,0.0,0.0,1.0,0.0,0.0,0.0,130,0.005305
3,1,4,0.999993,0.999525,1.0,0.000000,0.128269,0.298795,0.246979,0.000000,...,0.014060,0.026144,0.0,0.0,0.0,1.0,0.0,0.0,130,0.007958
4,1,5,0.595137,0.736698,0.0,0.238089,0.014130,0.037871,0.058152,0.293184,...,0.135460,0.143240,0.0,0.0,1.0,0.0,0.0,0.0,130,0.010610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53754,260,312,0.476188,0.831354,1.0,0.626985,0.672172,0.682297,0.591489,0.507937,...,0.486283,0.483993,1.0,0.0,0.0,0.0,0.0,0.0,4,0.824934
53755,260,313,0.238102,0.298100,1.0,0.597937,0.644830,0.733008,0.722934,0.617180,...,0.614540,0.622022,0.0,0.0,0.0,0.0,1.0,0.0,3,0.827586
53756,260,314,0.595222,0.736342,0.0,0.238089,0.017892,0.088067,0.082198,0.293184,...,0.137517,0.144474,0.0,0.0,1.0,0.0,0.0,0.0,2,0.830239
53757,260,315,0.595203,0.738717,0.0,0.238089,0.021195,0.079155,0.102368,0.293184,...,0.132716,0.134383,0.0,0.0,1.0,0.0,0.0,0.0,1,0.832891


In [86]:
# normalize the columns in the test data.
# MinMax normalization
td['cycle_norm'] = td['cycle']
cols_normalize = td.columns.difference(['id','cycle'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(td[cols_normalize]), 
                             columns=cols_normalize, 
                             index=td.index)
join_df = td[td.columns.difference(cols_normalize)].join(norm_train_df)
td = join_df.reindex(columns = td.columns)
td

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s19,s20,s21,mode1,mode2,mode3,mode4,mode5,mode6,cycle_norm
0,1,1,0.238019,0.297150,1.0,0.597937,0.639243,0.691329,0.680395,0.617180,...,1.0,0.625944,0.630724,0.0,0.0,0.0,0.0,1.0,0.0,0.000000
1,1,2,0.476162,0.831354,1.0,0.626985,0.664861,0.647519,0.534663,0.507937,...,1.0,0.484214,0.496301,1.0,0.0,0.0,0.0,0.0,0.0,0.002732
2,1,3,0.833282,0.997625,1.0,0.060269,0.189055,0.312512,0.243225,0.146592,...,1.0,0.164722,0.159476,0.0,0.0,0.0,0.0,0.0,1.0,0.005464
3,1,4,0.999967,0.998812,1.0,0.000000,0.135525,0.286753,0.242061,0.000000,...,1.0,0.005834,0.016866,0.0,0.0,0.0,1.0,0.0,0.0,0.008197
4,1,5,0.595089,0.737886,0.0,0.238089,0.012028,0.022001,0.050986,0.293184,...,0.0,0.141730,0.140123,0.0,0.0,1.0,0.0,0.0,0.0,0.010929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33986,259,119,0.833210,0.997981,1.0,0.060269,0.185015,0.331523,0.245972,0.146592,...,1.0,0.163349,0.159133,0.0,0.0,0.0,0.0,0.0,1.0,0.322404
33987,259,120,0.999967,0.998219,1.0,0.000000,0.128638,0.290703,0.232087,0.000000,...,1.0,0.013384,0.016260,0.0,0.0,0.0,1.0,0.0,0.0,0.325137
33988,259,121,0.999955,0.997625,1.0,0.000000,0.130750,0.285244,0.219835,0.000000,...,1.0,0.013384,0.020433,0.0,0.0,0.0,1.0,0.0,0.0,0.327869
33989,259,122,0.000057,0.000356,1.0,1.000000,0.984024,0.944915,0.935747,1.000000,...,1.0,0.991764,0.985410,0.0,1.0,0.0,0.0,0.0,0.0,0.330601


Keras LSTM layers expect an input in the shape of a numpy array of 3 dimensions (samples, time steps, features) where samples is the number of training sequences, time steps is the look back window or sequence length and features is the number of features of each sequence at each time step.

In [87]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191

In [88]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3','cycle_norm','mode1','mode2','mode3','mode4','mode5','mode6']
# sequence_cols = ['setting1', 'setting2', 'setting3','cycle_norm']
sequence_cols.extend(sensor_cols)

In [89]:
sequence_length = 20

In [90]:
# generator for the sequences
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) 
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array.shape

(48559, 20, 31)

In [91]:
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

In [92]:
# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) 
             for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape
label_array

array([[128.],
       [127.],
       [126.],
       ...,
       [  2.],
       [  1.],
       [  0.]], dtype=float32)

### LSTM Network
Next, we build a deep network. The first layer is an LSTM layer with 100 units followed by another LSTM layer with 50 units. Dropout is also applied after each LSTM layer to control overfitting. Final layer is a Dense output layer with single unit and sigmoid activation since this is a binary classification problem.

### USE the model in github

In [93]:
import keras.backend as K

In [94]:
def r2_keras(y_true, y_pred):
    """Coefficient of Determination 
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [95]:
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae',r2_keras])

print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 100)           52800     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 100)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 83,051
T

In [96]:
# fit the network
model_path='reg_oh_rul.h5'
history = model.fit(seq_array, label_array, epochs=100, batch_size=300, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path,monitor='val_loss', save_best_only=True, mode='min', verbose=0)]
          )

# list all data in history
print(history.history.keys())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 46131 samples, validate on 2428 samples
Epoch 1/100
 - 39s - loss: 6795.1525 - mean_absolute_error: 71.2121 - r2_keras: -2.6328e+00 - val_loss: 6475.7599 - val_mean_absolute_error: 69.8195 - val_r2_keras: -2.9621e+00
Epoch 2/100
 - 34s - loss: 5298.8748 - mean_absolute_error: 62.1310 - r2_keras: -1.8326e+00 - val_loss: 5060.1340 - val_mean_absolute_error: 61.3904 - val_r2_keras: -2.1396e+00
Epoch 3/100
 - 34s - loss: 4104.2416 - mean_absolute_error: 54.7020 - r2_keras: -1.1936e+00 - val_loss: 3906.5195 - val_mean_absolute_error: 54.2573 - val_r2_keras: -1.5223e+00
Epoch 4/100
 - 34s - loss: 3168.9554 - mean_absolute_error: 48.6393 - r2_keras: -6.9347e-01 - val_loss: 3017.5036 - val_mean_absolute_error: 48.4411 - val_r2_keras: -1.1109e+00
Epoch 5/100
 - 34s - loss: 2505.3670 - mean_absolute_error: 44.0872 - r2_keras: -3.3590e-01 - val_loss: 2392.1211 - val_mean_absolute_error: 43.9445

In [97]:
model.load_weights('reg_oh_rul.h5')
ans = model.predict(seq_array,verbose=1)
ans

48559/48559 [==============================] - 27s 560us/step


array([[126.20988 ],
       [126.265366],
       [126.26546 ],
       ...,
       [  7.471456],
       [  8.249744],
       [  8.484245]], dtype=float32)

In [98]:
# training metrics
scores = model.evaluate(seq_array, label_array, verbose=1, batch_size=200)
print('Loss: {}'.format(scores[0]))

48559/48559 [==============================] - 13s 262us/step
Loss: 477.6900233805662


### Calculate result

In [99]:
# We pick the last sequence for each id in the test data
test = [td[td['id']==id][sequence_cols].values[-sequence_length:] 
                       for id in td['id'].unique() if len(td[td['id']==id]) >= sequence_length]

test = np.asarray(test).astype(np.float32)

In [115]:
result = model.predict(test,verbose=1)
result

259/259 [==============================] - 0s 632us/step


array([[ 13.310577 ],
       [124.38631  ],
       [ 76.5647   ],
       [108.96397  ],
       [ 14.6176815],
       [117.81574  ],
       [ 15.837944 ],
       [ 28.576025 ],
       [  9.655753 ],
       [109.09859  ],
       [ 32.532124 ],
       [116.30535  ],
       [ 30.312172 ],
       [  8.277908 ],
       [ 49.57695  ],
       [ 93.86366  ],
       [ 89.7553   ],
       [122.31261  ],
       [ 24.720848 ],
       [ 61.048046 ],
       [ 65.82861  ],
       [ 32.99668  ],
       [121.55955  ],
       [ 61.838837 ],
       [ 85.06111  ],
       [106.038666 ],
       [114.43005  ],
       [123.00956  ],
       [100.46672  ],
       [ 55.516724 ],
       [ 89.24947  ],
       [ 13.14628  ],
       [ 44.131428 ],
       [119.86713  ],
       [104.654205 ],
       [ 86.02544  ],
       [115.0925   ],
       [ 30.590641 ],
       [ 35.545258 ],
       [ 64.62901  ],
       [ 33.191193 ],
       [ 38.33864  ],
       [ 63.975258 ],
       [ 17.539791 ],
       [ 11.292589 ],
       [10

In [101]:
result.shape

(259, 1)

---

### 生成 .csv 文件

In [104]:
import time

In [105]:
localtime = time.asctime( time.localtime(time.time()) )

In [109]:
submit = pd.read_excel('../results_submission.xls')
submit['RUL prediction']=result
submit.to_csv(str(localtime)+'.csv')

## 计分！

In [112]:
ans = np.loadtxt('../RUL_FD002.txt')
ans.shape

(259,)

In [113]:
def score(user_ans):
    user_ans_arr = np.array(user_ans)
    score = 0
    diff = user_ans_arr - ans
    diff_pos = diff[diff>=0]
    diff_neg = diff[diff<0]
    score = np.sum((np.exp(-1./13.*diff_neg)-1))+np.sum((np.exp(1./10.*diff_pos)-1))
    return score

In [121]:
re = pd.read_csv('./Thu Apr 16 11:45:00 2020.csv')
result1 = re['RUL prediction']

result[0]

array([13.310577], dtype=float32)

In [122]:
score(result1)

13896.116346888302